In [1]:
# import the necessary packages
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import MaxPooling2D, MaxPooling3D
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Activation, Softmax
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import concatenate, add

## Resnext

In [2]:
!pip install tf-nightly-gpu-2.0-preview

     |████████████████████████████████| 395.5MB 37kB/s 
     |████████████████████████████████| 450kB 47.8MB/s 
     |████████████████████████████████| 3.8MB 31.7MB/s 
     |████████████████████████████████| 81kB 10.4MB/s 
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.11.0 which is incompatible.
ERROR: tb-nightly 2.1.0a20191206 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
  Found existing installation: google-auth 1.4.2
    Uninstalling google-auth-1.4.2:
      Successfully uninstalled google-auth-1.4.2


In [0]:
# conv with batch normalization and relu
def redBR(x, K, kSize, padding="same"):
    
    # CONV -> BN -> RELU pattern
    x = Conv2D(K, (kSize, kSize), padding=padding)(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    # return the conv block
    return x

In [0]:
# conv with batch normalization
def redB(x, K, padding="same"):
    
    # CONV -> BN -> RELU pattern
    x = Conv2D(K, (1, 1), padding=padding)(x)
    x = BatchNormalization()(x)

    # return the conv block
    return x

In [0]:
def tower(x, K):

    # CONV 1x1, CONV3x3, CONV1x1
    x = redBR(x, K, 1)
    x = redBR(x, K, 3)
    x = redB(x, K)

    return x

In [0]:
def Conv_block(x, K):

    # Start with the conv 1x1 path
    output = redB(x, K)

    # Add in the 32 towers in parallel
    for i in range(32):
        output += tower(x, K)

    # relu
    output = Activation('relu')(output)

    return output

In [0]:
def Identity_block(x, K):

    # Start with the input
    output = x

    # Add in 32 towers in parallel
    for i in range(32):
        output += tower(x, K)

    # relu
    output = Activation('relu')(output)
    
    return output

In [0]:
def resnext(width, height, depth, classes, K=64):

    # initialize the input shape
    inputShape = (height, width, depth)
    
    # define the model input
    inputs = Input(shape=inputShape)

    # conv 7 x 7
    x = redBR(inputs, K, 7)

    # max pool 3 x 3
    x = MaxPooling2D((3, 3), padding='same')(x)

    # Conv block 1
    x = Conv_block(x, K)

    # 2 Identity blocks
    x = Identity_block(x, K)
    x = Identity_block(x, K)

    # Conv block 2
    x = Conv_block(x, K)

    # 3 Identity blocks
    x = Identity_block(x, K)
    x = Identity_block(x, K)
    x = Identity_block(x, K)

    # Conv block 3
    x = Conv_block(x, K)

    # 5 Identity blocks
    x = Identity_block(x, K)
    x = Identity_block(x, K)
    x = Identity_block(x, K)
    x = Identity_block(x, K)
    x = Identity_block(x, K)

    # Conv block 4
    x = Conv_block(x, K)

    # 2 Identity blocks
    x = Identity_block(x, K)
    x = Identity_block(x, K)

    # global avg-pool
    x = GlobalAveragePooling2D()(x)

    # fully connected
    x = Dense(units=classes, activation='softmax')(x)

    # create the model
    model = Model(inputs, x, name="resnext")

    # return the custom model
    return model

In [9]:
myResnext = resnext(256, 256, 3, 6)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [10]:
print(myResnext.summary())

Model: "resnext"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 256, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 256, 256, 64) 256         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 256, 256, 64) 0           batch_normalization[0][0]        
____________________________________________________________________________________________

## Train the Model

In [11]:
from google.colab import files
!pip install kaggle

In [0]:
!mkdir .kaggle

In [0]:
!mkdir ~/.kaggle

In [0]:
import json
token = {"username":"kylestormcloud","key":"33a932fc4db62fdb2bd8a9ea76b664ed"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

In [0]:
!chmod 600 /content/.kaggle/kaggle.json

In [0]:
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json

In [17]:
!kaggle config set -n path -v{/content}

- path is now set to: {/content}


In [18]:
!kaggle datasets download -d sriramr/apples-bananas-oranges -p /content

 97% 498M/512M [00:04<00:00, 61.8MB/s]
100% 512M/512M [00:04<00:00, 110MB/s] 


In [19]:
!unzip \*.zip

Archive:  apples-bananas-oranges.zip
  inflating: original_data_set/freshapples/Screen Shot 2018-06-08 at 4.59.36 PM.png  
  inflating: original_data_set/freshapples/Screen Shot 2018-06-08 at 4.59.44 PM.png  
  inflating: original_data_set/freshapples/Screen Shot 2018-06-08 at 4.59.49 PM.png  
  inflating: original_data_set/freshapples/Screen Shot 2018-06-08 at 4.59.57 PM.png  
  inflating: original_data_set/freshapples/Screen Shot 2018-06-08 at 5.00.03 PM.png  
  inflating: original_data_set/freshapples/Screen Shot 2018-06-08 at 5.00.12 PM.png  
  inflating: original_data_set/freshapples/Screen Shot 2018-06-08 at 5.00.18 PM.png  
  inflating: original_data_set/freshapples/Screen Shot 2018-06-08 at 5.00.26 PM.png  
  inflating: original_data_set/freshapples/Screen Shot 2018-06-08 at 5.00.35 PM.png  
  inflating: original_data_set/freshapples/Screen Shot 2018-06-08 at 5.00.43 PM.png  
  inflating: original_data_set/freshapples/Screen Shot 2018-06-08 at 5.00.50 PM.png  
  inflating: orig

In [0]:
!rm -rf /content/original_data_set/original_data_set

In [21]:
!ls

apples-bananas-oranges.zip  original_data_set  sample_data


In [0]:
import pathlib
data_dir = pathlib.Path('original_data_set')

In [23]:
from __future__ import absolute_import, division, print_function, unicode_literals

%tensorflow_version 2.x
import tensorflow as tf
AUTOTUNE = tf.data.experimental.AUTOTUNE
from tensorflow.keras import datasets, layers, models
import IPython.display as display
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os

TensorFlow is already loaded. Please restart the runtime to change versions.


In [24]:
image_count = len(list(data_dir.glob('*/*.png')))
image_count

1511

In [0]:
CLASS_NAMES = np.array([item.name for item in data_dir.glob('*') if item.name != '.ipynb_checkpoints'])

In [0]:
# Image Generator Constants Form
#@title Data Augmentation
rotation_range = 45 #@param {type:"slider", min:0, max:90, step:1}
width_shift_range =  0.15#@param {type:"number"}
height_shift_range = 0.15 #@param {type:"number"}
horizontal_flip = True #@param {type:"boolean"}
zoom_range = 0.1 #@param {type:"number"}

In [0]:
# The 1./255 is to convert from uint8 to float32 in range [0,1]. Split data into 80/20 
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                                                  rotation_range=rotation_range, # rotate 45 degrees
                                                                  width_shift_range=width_shift_range, # ?
                                                                  height_shift_range=height_shift_range, # ?
                                                                  horizontal_flip=horizontal_flip, # flip horizontally
                                                                  zoom_range=zoom_range,
                                                                  validation_split=0.2)

In [0]:
# CONSTANTS
#@title Constants
BATCH_SIZE = 64 #@param ["16", "32", "64", "128", "256"] {type:"raw"}
IMG_HEIGHT =  256 #@param {type:"integer"}
IMG_WIDTH = 256 #@param {type:"integer"}
STEPS_PER_EPOCH = np.ceil(image_count/BATCH_SIZE)

In [29]:
train_data_gen = image_generator.flow_from_directory(directory=str(data_dir),
                                                     batch_size=BATCH_SIZE,
                                                     shuffle=True,
                                                     subset = 'training',
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                     class_mode="sparse",
                                                     classes = list(CLASS_NAMES))
test_data_gen = image_generator.flow_from_directory(directory=str(data_dir),
                                                     batch_size=BATCH_SIZE,
                                                     shuffle=True,
                                                     subset = 'validation',
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                     class_mode="sparse",
                                                     classes = list(CLASS_NAMES))

Found 1212 images belonging to 6 classes.
Found 300 images belonging to 6 classes.


In [0]:
train_images, train_labels = next(train_data_gen)
test_images, test_labels = next(test_data_gen)

In [0]:
# Conpile parameters
#@title Compile the Model
optimizer = "adam" #@param ["adadelta", "adagrad", "adam", "adamax", "ftrl", "nadam", "rmsprop", "sgd"]
loss = "sparse_categorical_crossentropy" #@param ["binary crossentropy", "categorical crossentropy", "categorical hinge", "cosine similarity", "hinge", "huber", "mean absolute error", "mean absolute percentage error", "poisson", "sparse_categorical_crossentropy", "squared hinge"]

In [0]:
import datetime, os
myResnext.compile(optimizer=optimizer,
              loss=loss,
              metrics=['accuracy'])

#logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
#tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

history = myResnext.fit(train_images, train_labels, epochs=10, 
                    validation_data=(test_images, test_labels)) #,callbacks=[tensorboard_callback]


Train on 64 samples, validate on 64 samples
Epoch 1/10
